In [1]:
from torch import nn
import os
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

In [ ]:
dataSetDir = "../Brain_tumor_MRI_classification_CNN/Brain Tumor MRI"
print(os.listdir(dataSetDir))

In [ ]:
# Define the transformations
transform = transforms.Compose([
    transforms.Resize((150, 150)),
    transforms.ToTensor(),
])

# Load the dataset
train_dataset = datasets.ImageFolder(root=dataSetDir + '/Training', transform=transform)

# Create a DataLoader
train_loader = DataLoader(train_dataset, batch_size=10, shuffle=True, num_workers=2)

# Function to compute mean and std
def get_mean_std(loader):
    mean = 0.
    std = 0.
    total_images_count = 0
    for images, _ in loader:
        batch_size = images.size(0)  # Get the current batch size
        images = images.view(batch_size, images.size(1), -1)  # Reshape for easier computation
        mean += images.mean(2).sum(0)  # Sum the mean across the batch
        std += images.std(2).sum(0)    # Sum the std across the batch
        total_images_count += batch_size  # Count total images processed

    mean /= total_images_count  # Average the mean
    std /= total_images_count    # Average the std
    return mean, std

# Calculate and print mean and std
mean, std = get_mean_std(train_loader)
print(f"Mean: {mean}")
print(f"Std: {std}")


In [4]:
trainTransform = transforms.Compose([
    transforms.Resize((150,150)),
    transforms.RandomResizedCrop(150),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.ToTensor(),
    transforms.Normalize(mean=mean, std=std)
])

testTransform = transforms.Compose([
    transforms.Resize((150,150)),
    transforms.ToTensor(),
    transforms.Normalize(mean=mean, std=std)
])

In [ ]:
MRI = {}
MRI['train'] = datasets.ImageFolder(root=dataSetDir + '/Training', transform=trainTransform)
MRI['test'] = datasets.ImageFolder(root=dataSetDir + '/Testing', transform = testTransform)
MRI

In [6]:
trainLoader = DataLoader(dataset=MRI['train'], batch_size=32, shuffle=True, num_workers=2)
testLoader = DataLoader(dataset=MRI['test'], batch_size=32, shuffle=True, num_workers=2)


In [ ]:
classes = MRI['train'].classes
classes

In [ ]:
class_index = MRI['test'].class_to_idx
class_index

In [ ]:
image, label = next(iter(trainLoader))
image.shape

In [ ]:
import random
import matplotlib.pyplot as plt
from PIL import Image
# Plotting random images of each file(class)

for brain_tumor_class in classes : # Loop through each class
    
    # Accessing images in files 
    class_dir = os.path.join(dataSetDir + '/Training' , brain_tumor_class) # The image path of the rice class
    images_path = os.listdir(class_dir) # List of images
    random_images = random.sample(images_path , k = 5) # Random sample of images
    
    # Plotting some sample images
    fig , ax = plt.subplots(1 , 5 , figsize = (12 , 3)) 
    plt.suptitle(f'Sample from "{brain_tumor_class}" brain tumor class', fontsize = 14 
                 , bbox=dict(facecolor='#4A5759', alpha=0.7 , boxstyle='round,pad=0.3') , y = 0.9) # Title for each sample
    
    for i , random_image in enumerate(random_images) : # Loop for each image
        image_path = os.path.join(class_dir , random_image) # Creating image path
        image = Image.open(image_path) # Load image
        
        ax[i].imshow(image) # Plot image
        ax[i].axis('off') # Remove the axis
    
    plt.subplots_adjust( hspace = -0.2 , wspace = 0.1 )
    plt.show()
    print('\n')

In [ ]:
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"
device

In [ ]:
class brainTumorClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.convolutionLayer = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=16, kernel_size=(3,3), stride=1),
            nn.BatchNorm2d(num_features=16),
            nn.ReLU(),
            
            nn.Conv2d(in_channels=16, out_channels=32, kernel_size=(3,3), stride=1),
            nn.BatchNorm2d(num_features=32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(2,2), stride=2),
            
            nn.Conv2d(in_channels=32,out_channels=64, kernel_size=(3,3), stride=1),
            nn.BatchNorm2d(num_features=64),
            nn.ReLU()           
            
        )
        
        self.fullyConnectedLayer = nn.Sequential(
            nn.Linear(in_features=71*71*64, out_features=512),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, 4),
            nn.LogSoftmax(dim=1)
        )
    def forward(self, x):
        convolutionOut = self.convolutionLayer(x)
        flatenedConOut = torch.flatten(convolutionOut,1)
        return self.fullyConnectedLayer(flatenedConOut)
    
model = brainTumorClassifier()
model.to(device=device)


In [ ]:
from torchsummary import summary
summary(model, (3, 150, 150))


In [14]:
criterion = nn.NLLLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [ ]:
# number of epochs to train the model
n_epochs = 10


for epoch in range(n_epochs):
    # monitor training loss
    train_loss = 0.0
    train_accuracy = 0
    
    ###################
    # train the model #
    ###################
    model.train() # prep model for training
    for data, target in train_loader:
        if torch.cuda.is_available():
            data, target = data.cuda(), target.cuda()
        # clear the gradients of all optimized variables
        optimizer.zero_grad()
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model(data)
        # calculate the loss
        loss = criterion(output, target)
        # backward pass: compute gradient of the loss with respect to model parameters
        loss.backward()
        # perform a single optimization step (parameter update)
        optimizer.step()
        # update running training loss
        train_loss += loss.item()*data.size(0)
        #calculate accuracy
        ps = torch.exp(output)
        top_p, top_class = ps.topk(1, dim=1)
        equals = top_class == target.view(*top_class.shape)
        train_accuracy += torch.mean(equals.type(torch.FloatTensor)).item()
    
# calculate average loss over an epoch
    train_loss = train_loss/len(train_loader.dataset)

    print('Epoch: {} \tTraining Loss: {:.6f}'.format(
            epoch+1, 
            train_loss
            ))
    print(f"Train accuracy: {train_accuracy/len(train_loader):.3f}")

In [ ]:
# Checking Test Performence
test_accuracy = 0
model.eval() # prep model for evaluation
for data, target in testLoader:
    if torch.cuda.is_available():
        data, target = data.cuda(), target.cuda()
    # forward pass: compute predicted outputs by passing inputs to the model
    output = model(data)
    # calculate the loss
    loss = criterion(output, target)
    #calculate accuracy
    ps = torch.exp(output)
    top_p, top_class = ps.topk(1, dim=1)
    equals = top_class == target.view(*top_class.shape)
    test_accuracy += torch.mean(equals.type(torch.FloatTensor)).item()

print(f"Test accuracy: {test_accuracy/len(testLoader):.3f}")
